In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
from pylr2 import regress2
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf, col
from pyspark.sql import functions as F

import matplotlib.pyplot as plt

In [ ]:
from src.data import spark_postgis
from src import constants

In [ ]:
spark = spark_postgis.get_spark()

In [ ]:
sdf_all = spark.read.parquet((constants.RESULTS_PATH / "gedi_neighbors_nau_l24a").as_posix())
sdf_all.createOrReplaceTempView("shots_table")
sdf_all = spark.sql("SELECT *, ST_GeomFromWKB(t1_geometry) AS t1_geom, ST_GeomFromWKB(t2_geometry) AS t2_geom FROM shots_table")
sdf_all = sdf_all.drop("t1_geometry", "t2_geometry")
print(sdf_all.count())
sdf_all.show(3)

In [ ]:
sdf_intact_s1 = spark.read.parquet((constants.RESULTS_PATH / "intact_control_s1").as_posix())
sdf_intact_s2 = spark.read.parquet((constants.RESULTS_PATH / "intact_control_s2").as_posix())
sdf_intact = sdf_intact_s1.select(["t1_shot_number", "t2_shot_number"]).join(sdf_intact_s2, on=["t1_shot_number", "t2_shot_number"], how="inner")
print(sdf_intact.count())
sdf_intact.show(3)

In [ ]:
# Occasionally, we get sets of three shots with a disturbance between.
# Sometimes it is valid to count these as separate samples
# (e.g. s1a -- disturbance -- s1b -- s2,
# where the pair s1a-s2 is a treatment sample and s1b-s2 is a control sample).
# But other times, it's really two measurements of the same sample
# (e.g. s1a -- s1b -- disturbance -- s2, where s1a-s2 and s1b-s2 are both
# measurements of the same disturbance event).
# Just to be on the safe side, we can remove all the duplicates.
# This function should be run on the control and treatment sets separately.


def remove_duplicates(df):
    print(
        "Found {} s1 duplicates".format(
            len(df[df.duplicated(subset=["t1_shot_number"])])
        )
    )
    print(
        "Found {} s2 duplicates".format(
            len(df[df.duplicated(subset=["t2_shot_number"])])
        )
    )
    df = df.drop_duplicates(subset=["t1_shot_number"], keep="first")
    df = df.drop_duplicates(subset=["t2_shot_number"], keep="first")

    return df

In [ ]:
degrade_sdf = spark.read.parquet(
    (constants.RESULTS_PATH / "gedi_degradation_glad_0d").as_posix()
)
glad_df = gpd.GeoDataFrame(degrade_sdf.toPandas(), geometry="t2_geom").copy()
glad_df.loc[glad_df.control_disturbance > 0, "sample_grp"] = "control"
# Note: points may have a control disturbance as well as a measured disturbance.
# in that case, we include them in the treatment group; we don't care that they
# were also disturbed at another, unmeasured time.
glad_df.loc[glad_df.measured_disturbance > 0, "sample_grp"] = "treatment"
print(len(glad_df))
print(len(glad_df[glad_df["sample_grp"] == "treatment"]))
print(len(glad_df[glad_df["sample_grp"] == "control"]))
control_df = remove_duplicates(glad_df[glad_df["sample_grp"] == "control"])
control_df["sample_grp"] = "control"
treatment_df = remove_duplicates(glad_df[glad_df["sample_grp"] == "treatment"])
treatment_df["sample_grp"] = "treatment"
glad_df = pd.concat([control_df, treatment_df])
print(len(glad_df))
print(len(glad_df[glad_df["sample_grp"] == "treatment"]))
print(len(glad_df[glad_df["sample_grp"] == "control"]))
control_n = len(glad_df[glad_df["sample_grp"] == "control"])

In [ ]:
degrade_sdf = spark.read.parquet(
    (constants.RESULTS_PATH / "gedi_degradation_afc_2022").as_posix()
)
afc_df = gpd.GeoDataFrame(degrade_sdf.toPandas(), geometry="t2_geom").copy()
afc_df.loc[afc_df.control_disturbance > 0, "sample_grp"] = "control"
# Note: points may have a control disturbance as well as a measured disturbance.
# in that case, we include them in the treatment group; we don't care that they
# were also disturbed at another, unmeasured time.
afc_df.loc[afc_df.measured_disturbance > 0, "sample_grp"] = "treatment"
print(len(afc_df))
print(len(afc_df[afc_df["sample_grp"] == "treatment"]))
print(len(afc_df[afc_df["sample_grp"] == "control"]))
control_df = remove_duplicates(afc_df[afc_df["sample_grp"] == "control"])
control_df["sample_grp"] = "control"
treatment_df = remove_duplicates(afc_df[afc_df["sample_grp"] == "treatment"])
treatment_df["sample_grp"] = "treatment"
afc_df = pd.concat([control_df, treatment_df])
print(len(afc_df))
print(len(afc_df[afc_df["sample_grp"] == "treatment"]))
print(len(afc_df[afc_df["sample_grp"] == "control"]))

## Temporal distribution of shot pairs

In [ ]:
@udf(returnType=IntegerType())
def get_days(time_delta):
  return time_delta.days

def add_time_diff(_sdf):
  _sdf = _sdf.withColumn("time_diff", (_sdf["t2_absolute_time"] - _sdf["t1_absolute_time"]))
  _sdf = _sdf.withColumn("time_diff", get_days(col("time_diff")))
  return _sdf

In [ ]:
sdf_all = add_time_diff(sdf_all)
sdf_intact = add_time_diff(sdf_intact)
glad_df["time_diff"] = (glad_df.t2_absolute_time - glad_df.t1_absolute_time).dt.days
afc_df["time_diff"] = (afc_df.t2_absolute_time - afc_df.t1_absolute_time).dt.days

In [ ]:
# Remove shot pairs with zero days between shots
sdf_all = sdf_all.filter(sdf_all.time_diff != 0)
sdf_intact = sdf_intact.filter(sdf_intact.time_diff != 0)
print(sdf_all.count())

time_diffs = np.array(sdf_intact.select("time_diff").rdd.flatMap(lambda x: x).collect())

In [ ]:
def temporal_adjust(df_orig):
    bins = np.arange(0, 1200, 60)
    df_orig["time_diff_bin"] = pd.cut(df_orig["time_diff"], bins=bins)
    treatment_dist = df_orig[df_orig.sample_grp == "treatment"].groupby("time_diff_bin").size().reset_index(name="count")
    # print(treatment_dist)

    new_control = []
    new_treatment = []
    for i in range(0, len(treatment_dist)):
        bin, count = treatment_dist.iloc[i]["time_diff_bin"], treatment_dist.iloc[i]["count"]
        c = df_orig[(df_orig.sample_grp == "control") & (df_orig.time_diff_bin == bin)]
        t = df_orig[(df_orig.sample_grp == "treatment") & (df_orig.time_diff_bin == bin)]
        if len(t) != count:
            print(f"I am confused: {bin}")
        if len(c) < count:
            new_treatment.append(t.sample(len(c), replace=False))
            new_control.append(c)
        else:
            new_control.append(c.sample(count, replace=False))
            new_treatment.append(t)

    new_control = pd.concat(new_control)
    new_treatment = pd.concat(new_treatment)
    return pd.concat([new_control, new_treatment])

In [ ]:
def print_stats(df):
    print((df.t2_agbd_a0 - df.t1_agbd_a0).mean())
    print((df.t2_rh_98_a0 - df.t1_rh_98_a0).mean())
    print((df.t2_rh_50_a0 - df.t1_rh_50_a0).mean())
    print("")
    print(df.t1_agbd_a0.corr(df.t2_agbd_a0, method="pearson"))
    print(df.t1_rh_98_a0.corr(df.t2_rh_98_a0, method="pearson"))
    print(df.t1_rh_50_a0.corr(df.t2_rh_50_a0, method="pearson"))

glad_df_adjust = temporal_adjust(glad_df)
print("GLAD")
print_stats(glad_df_adjust[glad_df_adjust.sample_grp == "control"])
afc_df_adjust = temporal_adjust(afc_df)
print("AFC")
print_stats(afc_df_adjust[afc_df_adjust.sample_grp == "control"])

In [ ]:
from matplotlib.lines import Line2D
import seaborn as sns

datas = [afc_df, afc_df_adjust]

fig, ax = plt.subplots(nrows=3, ncols=2, figsize=(8,12))
bin_count = 20
for i, data in enumerate(datas):
    print(len(data[data.sample_grp == "control"]))
    print(len(data[data.sample_grp == "treatment"]))
    axi = ax[0, i]
    sns.histplot(data[data.sample_grp == "treatment"].t1_absolute_time, ax=axi, color="blue", bins=bin_count)
    sns.histplot(data[data.sample_grp == "treatment"].t2_absolute_time, ax=axi, color="red", bins=bin_count)
    axi.set_xlabel("Date")
    axi.set_title("Treatment")
    legend_elements = [
            Line2D([0], [0], marker='o', color='blue', label="Shot 1",
                markerfacecolor='blue', markersize=7, linestyle='None'),
            Line2D([0], [0], marker='o', color='red', label="Shot 2",
                markerfacecolor='red', markersize=7, linestyle='None'),
        ]
    axi.legend(handles=legend_elements)
    axi.xaxis.set_major_locator(plt.MaxNLocator(3))

    axi = ax[1, i]
    sns.histplot(data[data.sample_grp == "control"].t1_absolute_time, ax=axi, color="blue", bins=bin_count)
    sns.histplot(data[data.sample_grp == "control"].t2_absolute_time, ax=axi, color="red", bins=bin_count)
    axi.set_xlabel("Date")
    axi.set_title("Control")
    legend_elements = [
            Line2D([0], [0], marker='o', color='blue', label="Shot 1",
                markerfacecolor='blue', markersize=7, linestyle='None'),
            Line2D([0], [0], marker='o', color='red', label="Shot 2",
                markerfacecolor='red', markersize=7, linestyle='None'),
        ]
    axi.legend(handles=legend_elements)
    axi.xaxis.set_major_locator(plt.MaxNLocator(3))

    axi = ax[2, i]
    sns.histplot(
        (
        data[data.sample_grp == "control"].time_diff), ax=axi, color="green", bins=20)

    sns.histplot(data[data.sample_grp == "treatment"].time_diff, ax=axi, color="orange", bins=20)
    axi.set_xlabel("Time between shots (days)")
    legend_elements = [
            Line2D([0], [0], marker='o', color='green', label="Control",
                markerfacecolor='green', markersize=7, linestyle='None'),
            Line2D([0], [0], marker='o', color='orange', label="Treatment",
                markerfacecolor='orange', markersize=7, linestyle='None'),
        ]
    axi.legend(handles=legend_elements)


plt.tight_layout()
plt.show()

## Correlation and bias versus time, distance

In [ ]:
sdf_all = sdf_all.withColumn("proj", spark_postgis.get_utm_projection(col("t1_geom")))
sdf_all.createOrReplaceTempView("shots_table")

In [ ]:
# DO NOT RUN THIS CELL UNLESS YOU WANT TO RECOMPUTE THE DATAAAA
###############################################################

sdf_30m = spark.sql("""
    SELECT * 
    FROM shots_table 
    WHERE ST_DISTANCE(
        ST_Transform(t1_geom, "epsg:4326", proj),
        ST_Transform(t2_geom, "epsg:4326", proj)) < 30""")

# Write out and then reread back in ...
# For some reason spark seems to recompute this every time we try to use it o/w.
sdf_30m.write.mode("overwrite").format("geoparquet").save(
    "/maps/forecol/results/gedi_neighbors_nau_l24a_30m"
)


In [ ]:

sdf_30m = spark.read.parquet(
    "/maps/forecol/results/gedi_neighbors_nau_l24a_30m"
)

In [ ]:
sdf_30m = (sdf_30m
    .withColumn("agbd_diff", sdf_30m.t2_agbd_a0 - sdf_30m.t1_agbd_a0)
    .withColumn("rh98_diff", sdf_30m.t2_rh_98_a0 - sdf_30m.t1_rh_98_a0)
)
sdf_30m.select(F.mean("agbd_diff")).show()
sdf_30m.select(F.mean("rh98_diff")).show()
print(sdf_30m.corr("t1_agbd_a0", "t2_agbd_a0"))
print(sdf_30m.corr("t1_rh_98_a0", "t2_rh_98_a0"))


In [ ]:
from typing import List
from pyspark.sql.functions import mean

def get_stats(_sdf, lags: List[int]):
    lags.sort(reverse=True)
    var_agbd = []
    var_rh98 = []
    bias_agbd = []
    bias_rh98 = []
    counts = []
    for i in range(len(lags) - 1):
        _tmp = _sdf.filter(_sdf.time_diff >= lags[i+1]).filter(_sdf.time_diff < lags[i])
        n = _tmp.count()
        counts.append(n)
        print(f"{lags[i+1]}-{lags[i]}: {n}")
        if n == 0:
            continue
        var_agbd.append(_tmp.select(F.stddev(col("agbd_diff"))).collect()[0][0])
        var_rh98.append(_tmp.select(F.stddev(col("rh98_diff"))).collect()[0][0])
        bias_agbd.append(_tmp.select(F.mean(col("agbd_diff"))).collect()[0][0])
        bias_rh98.append(_tmp.select(F.mean(col("rh98_diff"))).collect()[0][0])
    return var_agbd, var_rh98, bias_agbd, bias_rh98, counts

In [ ]:
time_lags = [1100, 1000, 900, 800, 700, 600, 500, 450, 400, 350, 300, 250, 200, 150, 100, 50, 0]
sdf_all = sdf_all.withColumn("agbd_diff", col("t2_agbd_a0") - col("t1_agbd_a0"))
sdf_all = sdf_all.withColumn("rh98_diff", col("t2_rh_98_a0") - col("t1_rh_98_a0"))
sdf_intact = sdf_intact.withColumn("agbd_diff", col("t2_agbd_a0") - col("t1_agbd_a0"))
sdf_intact = sdf_intact.withColumn("rh98_diff", col("t2_rh_98_a0") - col("t1_rh_98_a0"))

# Uncomment to also compute stats at 30 m apart.
# This takes a long time.

# sdf_30m = sdf_30m.withColumn("agbd_diff", col("t2_agbd_a0") - col("t1_agbd_a0"))
# sdf_30m = sdf_30m.withColumn("rh98_diff", col("t2_rh_98_a0") - col("t1_rh_98_a0"))
# var_agbd, var_rh98, bias_agbd, bias_rh98, counts = get_stats(sdf_all, time_lags)
# var_agbd_30m, var_rh98_30m, bias_agbd_30m, bias_rh98_30m, counts_30m = get_stats(sdf_30m, time_lags)
# print(bias_agbd)
# print(counts)


In [ ]:
from matplotlib.lines import Line2D
n_lags = len(time_lags) - 1
lags_text = [f"[{time_lags[n_lags - i]},{time_lags[n_lags - (i+1)]})" for i in range(n_lags)]
x_lags = time_lags[1:]
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10,10))
axi = ax[0,0]
axi.scatter(x=x_lags, y=var_agbd, color="darkorange")
axi.scatter(x=x_lags, y=var_agbd_30m, color="bisque")
axi.set_xlabel("Time between shots (days)")
axi.set_ylabel("Standard deviation")
axi.set_title("AGBD: Time lag vs std deviation")
axi.set_xticks(x_lags, lags_text, rotation=45)
legend_elements = [
        Line2D([0], [0], marker='o', color='w', label='< 30 m apart',
            markerfacecolor='bisque', markersize=5),
        Line2D([0], [0], marker='o', color='w', label='< 40 m apart', 
            markerfacecolor='darkorange', markersize=5),]
axi.legend(handles=legend_elements)


axi = ax[0,1]
axi.scatter(x=x_lags, y=var_rh98, color="blue")
axi.scatter(x=x_lags, y=var_rh98_30m, color="lightsteelblue")
axi.set_xlabel("Time between shots (days)")
axi.set_title("RH 98: Time lag vs std deviation")
legend_elements = [
        Line2D([0], [0], marker='o', color='w', label='< 30 m apart',
            markerfacecolor='lightsteelblue', markersize=5),
        Line2D([0], [0], marker='o', color='w', label='< 40 m apart', 
            markerfacecolor='blue', markersize=5),]
axi.legend(handles=legend_elements)

axi = ax[1,0]
axi.scatter(x=x_lags, y=bias_agbd, color="darkorange")
axi.scatter(x=x_lags, y=bias_agbd_30m, color="bisque")
axi.set_xlabel("Time between shots (days)")
axi.set_ylabel("Bias (mean difference)")
axi.set_title("AGBD: Time lag vs Bias")
legend_elements = [
        Line2D([0], [0], marker='o', color='w', label='< 30 m apart',
            markerfacecolor='bisque', markersize=5),
        Line2D([0], [0], marker='o', color='w', label='< 40 m apart', 
            markerfacecolor='darkorange', markersize=5),]
axi.legend(handles=legend_elements)


axi = ax[1,1]
axi.scatter(x=x_lags, y=bias_rh98, color="blue")
axi.scatter(x=x_lags, y=bias_rh98_30m, color="lightsteelblue")
axi.set_xlabel("Time between shots (days)")
axi.set_title("RH 98: Time lag vs Bias")
legend_elements = [
        Line2D([0], [0], marker='o', color='w', label='< 30 m apart',
            markerfacecolor='lightsteelblue', markersize=5),
        Line2D([0], [0], marker='o', color='w', label='< 40 m apart', 
            markerfacecolor='blue', markersize=5),]
axi.legend(handles=legend_elements)

In [ ]:
def get_stats_pandas(df, lags):
    lags.sort(reverse=True)
    var_agbd = []
    var_rh98 = []
    bias_agbd = []
    bias_rh98 = []
    counts = []
    print(lags)
    for i in range(len(lags) - 1):
        _tmp = df[(df.time_diff >= lags[i+1]) & (df.time_diff < lags[i])]
        n = len(_tmp)
        counts.append(n)
        print(f"{lags[i+1]}-{lags[i]}: {n}")
        if n < 40:
            var_agbd.append(None)
            var_rh98.append(None)
            bias_agbd.append(None)
            bias_rh98.append(None)
            continue
        var_agbd.append(_tmp["agbd_diff"].std())
        var_rh98.append(_tmp["rh98_diff"].std())
        bias_agbd.append(_tmp["agbd_diff"].mean())
        bias_rh98.append(_tmp["rh98_diff"].mean())
    return var_agbd, var_rh98, bias_agbd, bias_rh98, counts


In [ ]:
time_lags = [1200, 1000, 900, 800, 700, 600, 500, 400, 300, 200, 100, 50, 0]
afc_df["agbd_diff"] = afc_df.t2_agbd_a0 - afc_df.t1_agbd_a0
afc_df["rh98_diff"] = afc_df.t2_rh_98_a0 - afc_df.t1_rh_98_a0
glad_df["agbd_diff"] = glad_df.t2_agbd_a0 - glad_df.t1_agbd_a0
glad_df["rh98_diff"] = glad_df.t2_rh_98_a0 - afc_df.t1_rh_98_a0


In [ ]:

afc_stats = get_stats_pandas(afc_df[afc_df.sample_grp == "control"], time_lags)
glad_stats = get_stats_pandas(glad_df[glad_df.sample_grp == "control"], time_lags)
all_stats = get_stats(sdf_all, time_lags)
intact_stats = get_stats(sdf_intact, time_lags)

In [ ]:
time_lags = [1200, 1000, 900, 800, 700, 600, 500, 400, 300, 200, 100, 50, 0]
glad_stats = get_stats_pandas(glad_df[glad_df.sample_grp == "control"], time_lags)
print(time_lags)
for s in glad_stats:
    print(s)


In [ ]:
from matplotlib.lines import Line2D
n_lags = len(time_lags) - 1
lags_text = [f"[{time_lags[n_lags - i]},{time_lags[n_lags - (i+1)]})" for i in range(n_lags)]
x_lags = time_lags[1:]

lms = 10
legend_elements = [
        Line2D([0], [0], marker='o', color='w', label='AFC Disturbed control',
            markerfacecolor='tab:red', markersize=lms),
        Line2D([0], [0], marker='o', color='w', label='GLAD Disturbed control', 
            markerfacecolor='tab:pink', markersize=lms),
        Line2D([0], [0], marker='o', color='w', label='All shots', 
            markerfacecolor='tab:orange', markersize=lms),
        Line2D([0], [0], marker='o', color='w', label='Intact shots', 
            markerfacecolor='tab:green', markersize=lms),
]

fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10,10))
axi = ax[0,0]
axi.plot(x_lags, afc_stats[0], '-o', color="tab:red")
axi.plot(x_lags, glad_stats[0], '-o', color="tab:pink")
axi.plot(x_lags, all_stats[0], '-o', color="tab:orange")
axi.plot(x_lags, intact_stats[0], '-o', color="tab:green")
axi.set_xlabel("Time between shots (days)")
axi.set_ylabel("Standard deviation")
axi.set_title("AGBD: Time lag vs std deviation")
axi.set_xticks(x_lags[::-1], lags_text, rotation=45)


axi = ax[0,1]
axi.plot(x_lags, afc_stats[1], '-o', color="tab:red")
axi.plot(x_lags, glad_stats[1], '-o', color="tab:pink")
axi.plot(x_lags, all_stats[1], '-o', color="tab:orange")
axi.plot(x_lags, intact_stats[1], '-o', color="tab:green")
axi.set_xlabel("Time between shots (days)")
axi.set_title("RH 98: Time lag vs std deviation")
axi.set_xticks(x_lags[::-1], lags_text, rotation=45)

axi = ax[1,0]
axi.plot(x_lags, afc_stats[2], '-o', color="tab:red")
axi.plot(x_lags, glad_stats[2], '-o', color="tab:pink")
axi.plot(x_lags, all_stats[2], '-o', color="tab:orange")
axi.plot(x_lags, intact_stats[2], '-o', color="tab:green")
axi.set_xlabel("Time between shots (days)")
axi.set_ylabel("Bias (mean difference)")
axi.set_title("AGBD (Mg/ha): Time lag vs Bias")
axi.set_xticks(x_lags[::-1], lags_text, rotation=45)


axi = ax[1,1]
axi.plot(x_lags, afc_stats[3], '-o', color="tab:red")
axi.plot(x_lags, glad_stats[3], '-o', color="tab:pink")
axi.plot(x_lags, all_stats[3], '-o', color="tab:orange")
axi.plot(x_lags, intact_stats[3], '-o', color="tab:green")
axi.set_xlabel("Time between shots (days)")
axi.set_title("RH 98 (m): Time lag vs Bias")
axi.set_xticks(x_lags[::-1], lags_text, rotation=45)

fig.subplots_adjust(wspace=0.30, hspace=0.5)
fig.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, 0.0),fancybox=False, shadow=False, ncol=4)
plt.tight_layout()

In [ ]:
# Temporal buffering
buffers = [i * 30 for i in range(0, 7)]
files = [f"gedi_degradation_glad_{i}d" for i in buffers]
paths = [constants.RESULTS_PATH / f for f in files]
print(paths)

In [ ]:
control_bias_glad = []
for path in paths:
    sdf = spark.read.parquet(path.as_posix())
    bias_agbd = (
        sdf.filter("measured_disturbance == 0")
        #  .where("t2_days < p1_disturb_date OR t2_days < p2_disturb_date OR t2_days < p3_disturb_date OR t2_days < p4_disturb_date OR t2_days < p5_disturb_date OR t2_days < p6_disturb_date OR t2_days < p7_disturb_date OR t2_days < p8_disturb_date OR t2_days < p9_disturb_date")
        #  .where("t2_yydoy < p1_disturb_date OR t2_yydoy < p2_disturb_date OR t2_yydoy < p3_disturb_date OR t2_yydoy < p4_disturb_date OR t2_yydoy < p5_disturb_date OR t2_yydoy < p6_disturb_date OR t2_yydoy < p7_disturb_date OR t2_yydoy < p8_disturb_date OR t2_yydoy < p9_disturb_date")
        #  .where("t1_yydoy > p1_disturb_date OR t1_yydoy > p2_disturb_date OR t1_yydoy > p3_disturb_date OR t1_yydoy > p4_disturb_date OR t1_yydoy > p5_disturb_date OR t1_yydoy > p6_disturb_date OR t1_yydoy > p7_disturb_date OR t1_yydoy > p8_disturb_date OR t1_yydoy > p9_disturb_date")
        .withColumn("agbd_diff", sdf["t2_agbd_a0"] - sdf["t1_agbd_a0"])
        .agg({"agbd_diff": "mean"})
        .collect()[0][0]
    )
    control_bias_glad.append(bias_agbd)
print(control_bias_glad)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
months = [b / 30 for b in buffers]
ax.plot(months, control_bias_glad, "-o", color="tab:orange")
ax.set_ylim(-15, 0)
ax.set_xlabel("Temporal buffer (months)")
ax.set_ylabel("Bias (Mg/ha)")